In [ ]:
import asyncio
import atexit
import threading
import time
from typing import List
from enum import Enum

import grpc
from google.protobuf.empty_pb2 import Empty

from reachy_sdk_api import camera_reachy_pb2_grpc
from reachy_sdk_api import joint_pb2, joint_pb2_grpc
from reachy_sdk_api import fan_pb2_grpc
from reachy_sdk_api import sensor_pb2, sensor_pb2_grpc
from reachy_sdk_api import restart_signal_pb2, restart_signal_pb2_grpc

from reachy_sdk_api.gripperMX28_pb2 import GripperMX28Command, GrippersMX28Command
from reachy_sdk_api.gripper_pb2 import GripperCommand, GripperId, GrippersAck
from reachy_sdk_api.gripperMX28_pb2_grpc import GripperMX28ServiceStub

In [ ]:
grpc_channel = grpc.insecure_channel(f'10.42.0.1:50055')

Gripper stuff

In [ ]:
gripper_stub = GripperMX28ServiceStub(grpc_channel)

## Checking compliance

In [ ]:
for i in range(50) :
    gripper_stub.SendGripperMX28Commands(GrippersMX28Command(commands=[GripperMX28Command(id=GripperId.RIGHT, goal_position=0.35)]))
    time.sleep(0.01)

In [ ]:
reachy.head.look_at(0.5,-0.3,0.0,duration=3.0)

Lift both arms, put them in stiff mode and check if *turn_off_smoothly* works.

In [ ]:
reachy.turn_on('reachy')

In [ ]:
reachy.turn_off_smoothly('reachy')

## Moving the arms

Put each arm at 90 degrees.

In [ ]:
try:
    right_angled_position_right = {
    reachy.r_arm.r_shoulder_pitch: 0,
    reachy.r_arm.r_shoulder_roll: 0,
    reachy.r_arm.r_arm_yaw: 0,
    reachy.r_arm.r_elbow_pitch: -90,
    reachy.r_arm.r_forearm_yaw: 0,
    reachy.r_arm.r_wrist_pitch: 0,
    reachy.r_arm.r_wrist_roll: 0,
}
except AttributeError:
    pass

try:
    right_angled_position_left = {
        reachy.l_arm.l_shoulder_pitch: 0,
        reachy.l_arm.l_shoulder_roll: 0,
        reachy.l_arm.l_arm_yaw: 0,
        reachy.l_arm.l_elbow_pitch: -90,
        reachy.l_arm.l_forearm_yaw: 0,
        reachy.l_arm.l_wrist_pitch: 0,
        reachy.l_arm.l_wrist_roll: 0,
    }
except AttributeError:
    pass

In [ ]:
reachy.turn_on('reachy')

In [ ]:
try:
    goto(
        goal_positions=right_angled_position_right,
        duration=1.0,
        interpolation_mode=InterpolationMode.LINEAR
    )
    goto(
        goal_positions=right_angled_position_left,
        duration=1.0,
        interpolation_mode=InterpolationMode.MINIMUM_JERK
    )
except NameError:
    print('Reachy has no right arm!')

In [ ]:
try:
    goto(
        goal_positions=right_angled_position_left,
        duration=1.0,
        interpolation_mode=InterpolationMode.MINIMUM_JERK
    )
except NameError:
    print('Reachy has no left arm!')

In [ ]:
reachy.turn_off_smoothly('reachy')

Reproduce the square movement from [the documentation](https://pollen-robotics.github.io/reachy-2021-docs/sdk/first-moves/kinematics/#inverse-kinematics), to check if this Reachy can reproduce it.

For the right arm:

In [ ]:
A = np.array([
  [0, 0, -1, 0.3],
  [0, 1, 0, -0.4],  
  [1, 0, 0, -0.3],
  [0, 0, 0, 1],  
])

B = np.array([
  [0, 0, -1, 0.3],
  [0, 1, 0, -0.4],  
  [1, 0, 0, 0.0],
  [0, 0, 0, 1],  
])

C = np.array([
  [0, 0, -1, 0.3],
  [0, 1, 0, -0.1],  
  [1, 0, 0, 0.0],
  [0, 0, 0, 1],  
])

D = np.array([
  [0, 0, -1, 0.3],
  [0, 1, 0, -0.1],  
  [1, 0, 0, -0.3],
  [0, 0, 0, 1],  
])

In [ ]:
try:
    joint_pos_A = reachy.r_arm.inverse_kinematics(A)
    joint_pos_B = reachy.r_arm.inverse_kinematics(B)
    joint_pos_C = reachy.r_arm.inverse_kinematics(C)
    joint_pos_D = reachy.r_arm.inverse_kinematics(D)
except AttributeError:
    print('Reachy has no right arm!')

In [ ]:
import time
# put the joints in stiff mode
try:
    reachy.turn_on('r_arm')

    # use the goto function
    goto({joint: pos for joint,pos in zip(reachy.r_arm.joints.values(), joint_pos_A)}, duration=1.0)
    time.sleep(0.5)
    goto({joint: pos for joint,pos in zip(reachy.r_arm.joints.values(), joint_pos_B)}, duration=1.0)
    time.sleep(0.5)
    goto({joint: pos for joint,pos in zip(reachy.r_arm.joints.values(), joint_pos_C)}, duration=1.0)
    time.sleep(0.5)
    goto({joint: pos for joint,pos in zip(reachy.r_arm.joints.values(), joint_pos_D)}, duration=1.0)

    # put the joints back to compliant mode
    # use turn_off_smoothly to prevent the arm from falling hard
    reachy.turn_off_smoothly('r_arm')
    
except AttributeError:
    print('Reachy has no right arm!')

For the left arm:

In [ ]:
A = np.array([
  [0, 0, -1, 0.3],
  [0, 1, 0, 0.1],  
  [1, 0, 0, -0.3],
  [0, 0, 0, 1],  
])

B = np.array([
  [0, 0, -1, 0.3],
  [0, 1, 0, 0.1],  
  [1, 0, 0, 0.0],
  [0, 0, 0, 1],  
])

C = np.array([
  [0, 0, -1, 0.3],
  [0, 1, 0, 0.3],  
  [1, 0, 0, 0.0],
  [0, 0, 0, 1],  
])

D = np.array([
  [0, 0, -1, 0.3],
  [0, 1, 0, 0.3],  
  [1, 0, 0, -0.3],
  [0, 0, 0, 1],  
])

In [ ]:
try:
    joint_pos_A = reachy.l_arm.inverse_kinematics(A)
    joint_pos_B = reachy.l_arm.inverse_kinematics(B)
    joint_pos_C = reachy.l_arm.inverse_kinematics(C)
    joint_pos_D = reachy.l_arm.inverse_kinematics(D)
except AttributeError:
    print('Reachy has no left arm')

In [ ]:
# put the joints in stiff mode
try:
    reachy.turn_on('l_arm')

    # use the goto function
    goto({joint: pos for joint,pos in zip(reachy.l_arm.joints.values(), joint_pos_A)}, duration=1.0)
    time.sleep(0.5)
    goto({joint: pos for joint,pos in zip(reachy.l_arm.joints.values(), joint_pos_B)}, duration=1.0)
    time.sleep(0.5)
    goto({joint: pos for joint,pos in zip(reachy.l_arm.joints.values(), joint_pos_C)}, duration=1.0)
    time.sleep(0.5)
    goto({joint: pos for joint,pos in zip(reachy.l_arm.joints.values(), joint_pos_D)}, duration=1.0)

    # put the joints back to compliant mode
    # use turn_off_smoothly to prevent the arm from falling hard
    reachy.turn_off_smoothly('l_arm')
except AttributeError:
    print('Reachy has no left arm!')

## Checking the force sensors

You should see one force sensor per arm, zith the *.force_sensors* object.

In [ ]:
reachy.force_sensors

The force value should be positive when applying a force towards the exterior of the gripper (e.g. when you try to open the gripper which is in stiff mode). On the contrary, you should get a negative force value when applying a force towards the interior of the gripper.  

First, place the grippers in stiff mode, it will be easier to check the force values.

In [ ]:
try:
    reachy.l_arm.l_gripper.compliant = False
except AttributeError:
    print('Reachy has no left arm.')

In [ ]:
try:
    reachy.r_arm.r_gripper.compliant = False
except AttributeError:
    print('Reachy has no right arm.')

Now, try to close each gripper and check that when you doing this, the force value is negative.

In [ ]:
reachy.force_sensors.r_force_gripper

In [ ]:
reachy.force_sensors.l_force_gripper

Next, try to open each gripper and check that when you doing this, the force value is positive.

In [ ]:
reachy.force_sensors.r_force_gripper

In [ ]:
reachy.force_sensors.l_force_gripper

Finally, put the grippers back to compliant mode.

In [ ]:
try:
    reachy.l_arm.l_gripper.compliant = True
except AttributeError:
    pass

In [ ]:
try:
    reachy.r_arm.r_gripper.compliant = True
except AttributeError:
    pass